In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import pandas as pd
import isodate  # Used to convert ISO 8601 duration format to HH:MM:SS

# Define OAuth 2.0 scope (Read-only access to YouTube)
SCOPES = ["https://www.googleapis.com/auth/youtube.readonly"]

# Authenticate using OAuth 2.0
flow = InstalledAppFlow.from_client_secrets_file("client_secret.json", SCOPES)
creds = flow.run_local_server(port=0)

# Build YouTube API client
youtube = build("youtube", "v3", credentials=creds)

# Function to get Uploads Playlist ID for the channel
def get_uploads_playlist_id():
    request = youtube.channels().list(
        part="contentDetails",
        id="UCVjCW3tx7MT24MikTJXOZIw"  # Your Fixed Channel ID
    )
    response = request.execute()

    if "items" not in response or not response["items"]:
        raise Exception("❌ Error: No channel data found! Double-check the Channel ID.")

    return response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

# Function to fetch all videos with additional details
def get_videos():
    playlist_id = get_uploads_playlist_id()
    videos = []
    next_page_token = None

    while True:
        request = youtube.playlistItems().list(
            part="snippet",
            playlistId=playlist_id,  # Fetch videos from Uploads playlist
            maxResults=50,
            pageToken=next_page_token
        )
        response = request.execute()

        video_ids = [item["snippet"]["resourceId"]["videoId"] for item in response.get("items", [])]

        # Fetch video details (duration, views, likes, comments)
        details_request = youtube.videos().list(
            part="contentDetails,statistics,snippet",
            id=",".join(video_ids)  # Fetch details for multiple videos in one call
        )
        details_response = details_request.execute()

        for item in details_response.get("items", []):
            title = item["snippet"]["title"]
            video_id = item["id"]
            video_url = f"https://www.youtube.com/watch?v={video_id}"
            duration = isodate.parse_duration(item["contentDetails"]["duration"])  # Convert ISO duration to HH:MM:SS
            published_at = item["snippet"]["publishedAt"]
            view_count = item["statistics"].get("viewCount", "0")
            like_count = item["statistics"].get("likeCount", "0")
            comment_count = item["statistics"].get("commentCount", "0")

            videos.append([title, video_url, str(duration), published_at, view_count, like_count, comment_count])

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break

    return videos

# Fetch videos
video_list = get_videos()

# Save to Excel
df = pd.DataFrame(video_list, columns=["Title", "URL", "Duration", "Published Date", "Views", "Likes", "Comments"])
df.to_excel("YouTube_Video_List.xlsx", index=False)

print("✅ YouTube video list with additional details saved as 'YouTube_Video_List.xlsx'")


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=253406904901-shu0pjqhddobn21msfjn1o7ue5q3k020.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53512%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=GaOcs8zIwN8GeWhph3guJOiaSy3ZXL&access_type=offline
✅ YouTube video list with additional details saved as 'YouTube_Video_List.xlsx'
